Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS307.N11

FINAL PROJECT - 31/10/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rteQsEdImArTjUqQPw6s6DoVadY2_dht?usp=sharing)

# **FINAL PROJECT - JOB RECOMMENDATION SYSTEM**

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time
import datetime

from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

sns.set()

In [ ]:
PATH = '/content/drive/MyDrive/Dataset/DS307.N11/Final Project/'

## Crawl job data from [TopCV](https://www.topcv.vn/tim-viec-lam-moi-nhat?sort=top_related)

In [ ]:
url_TopCV = 'https://www.topcv.vn/tim-viec-lam-moi-nhat?sort=top_related'

In [ ]:
TopCV = requests.get(url_TopCV).content
print(TopCV)

b'<!DOCTYPE html>\n<html lang="vi">\n<head>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<link rel="icon" href="https://www.topcv.vn/v3/images/new/favicon.ico" type="image/x-icon">\n<meta property="fb:app_id" content="1478418029113221" />\n<meta name="csrf-token" content="lr7uasJxNr6A0JOAJgea2kzEU8pTmRXdSL096YRy">\n<meta name="google-site-verification" content="okX9WYRRKSr_EuYuN-C58UPMnAAusti-V-UFcpJEVZg" />\n<link rel="preconnect" href="https://www.googletagmanager.com">\n<link rel="preconnect" href="https://analytics.tiktok.com">\n<link rel="preconnect" href="https://www.googleadservices.com">\n<link rel="preconnect" href="https://www.google-analytics.com">\n<link rel="preconnect" href="https://connect.facebook.net">\n<link rel="preconnect" href="https://www.gstatic.com">\n<link rel="canonical" href="https://www.topcv.vn/tim-viec-lam-moi-nhat" />\n<link rel="publisher" 

In [ ]:
soup_TopCV = BeautifulSoup(TopCV, 'html.parser')
print(soup_TopCV)

<!DOCTYPE html>

<html lang="vi">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://www.topcv.vn/v3/images/new/favicon.ico" rel="icon" type="image/x-icon"/>
<meta content="1478418029113221" property="fb:app_id">
<meta content="lr7uasJxNr6A0JOAJgea2kzEU8pTmRXdSL096YRy" name="csrf-token"/>
<meta content="okX9WYRRKSr_EuYuN-C58UPMnAAusti-V-UFcpJEVZg" name="google-site-verification">
<link href="https://www.googletagmanager.com" rel="preconnect"/>
<link href="https://analytics.tiktok.com" rel="preconnect"/>
<link href="https://www.googleadservices.com" rel="preconnect"/>
<link href="https://www.google-analytics.com" rel="preconnect"/>
<link href="https://connect.facebook.net" rel="preconnect"/>
<link href="https://www.gstatic.com" rel="preconnect"/>
<link href="https://www.topcv.vn/tim-viec-lam-moi-nhat" rel="canonical">
<link href="https://plus.google.com

### Crawl url page

In [ ]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_TCV(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://www.topcv.vn/tim-viec-lam-moi-nhat?salary=0&exp=0&sort=top_related&page='+str(i))
  return list_url_page

In [ ]:
# Lấy danh sách các page có danh sách các công việc
list_url_page = get_url_page_TCV(400)

### Crawl url job

In [ ]:
# Viết hàm lấy url job của các công việc
def get_url_job_TCV(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_job = []
  # Lấy nội dung của page
  Content = requests.get(url_page).content
  Soup = BeautifulSoup(Content, 'html.parser')
  # Lấy url job
  url_jobs = Soup.find('div', {'class': 'lists'}).find_all('a', href=True, class_='underline-box-job')
  for job in url_jobs:
    list_url_job.append(job['href'])
  
  return list_url_job

In [ ]:
list_url_job_TCV = []
for url_page in tqdm(list_url_page):
  try:
    list_url_job_TCV.extend(get_url_job_TCV(url_page))
  except:
    print(f'{url_page} -> Error')

100%|██████████| 400/400 [10:15<00:00,  1.54s/it]


In [ ]:
list_url_job_TCV = list(set(list_url_job_TCV))
len(list_url_job_TCV)

9985

In [ ]:
df_url_job_TCV = pd.DataFrame({'URL Job': list_url_job_TCV})
df_url_job_TCV

,URL Job
0,https://www.topcv.vn/viec-lam/sale-admin-websi...
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...
2,https://www.topcv.vn/viec-lam/hr-business-part...
3,https://www.topcv.vn/viec-lam/general-manager/...
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...
...,...
9980,https://www.topcv.vn/viec-lam/nhan-vien-sales-...
9981,https://www.topcv.vn/viec-lam/chinese-speaking...
9982,https://www.topcv.vn/viec-lam/data-engineer/88...
9983,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...


In [ ]:
df_url_job_TCV.to_csv(PATH+'/job_url_TCV.csv',index_label='ID')

### Crawl job data

| Job Title | Name Company | Company Overview | Company Size | Company Address | Job Description | Job Requirements | Benefits | Job Address | Job Type | Gender | Number Cadidate | Career Level | Years of Experience | Salary | Submission Deadline | Industry |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|

In [ ]:
def crawl_job_data_TCV1(url):
  content_job = requests.get(url).content
  soup_job = BeautifulSoup(content_job, 'html.parser')
  # Tiêu đề tuyển dụng
  job_title = soup_job.find('h1', {'class':'job-title text-highlight bold'}).text.strip()
  # Tên công ty
  name_company = soup_job.find('div', {'class':'company-title'}).text.strip()
  # Giới thiệu công ty
  infor_company = soup_job.find('div', {'class':'box-info-company box-white'}).find('div', {'class':'box-info'}).find_all('span', {'class':'content'})
  infor_company_new = []
  for text in infor_company:
    infor_company_new.append(text.text)

  intro_company = infor_company_new[0]
  # Quy mô công ty
  size_company = infor_company_new[1]
  # Địa điểm công ty
  address_company = infor_company_new[2]
  # Thông tin chung
  general = soup_job.find('div', {'class':'box-main'}).text.strip()
  general = re.sub('\n', ' ', general)
  general = re.split('   ', general)
  general = list(filter(lambda x: x != '', general))

  for i in range(len(general)):
    general[i] = general[i].strip()

  # Mức lương
  Salary = general[1]
  # Số lượng tuyển
  NumCadidate = general[3]
  # Hình thức làm việc
  HinhThuc = general[5]
  # Cấp bậc
  level = general[7]
  # Giới tính 
  Gender = general[9]
  # Kinh nghiệm
  Experiment = general[11]
  # Mô tả công việc
  JD1 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[0].text.strip()
  jd1 = re.sub('\n+', '. ', JD1)
  # Yêu cầu ứng viên
  JD2 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[1].text.strip()
  jd2 = re.sub('\n+', '. ', JD2)
  # Quyền lợi
  JD3 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[2].text.strip()
  jd3 = re.sub('\n+', '. ', JD3)
  # Hạn nộp hồ sơ
  date = soup_job.find('div', {'class':'box-how-to-apply'}).text.strip()[-10:]
  # Ngành nghề
  NganhNghe = soup_job.find('div', {'class':'keyword'}).text.strip()
  NganhNghe = re.split('\n\n\n', NganhNghe)
  # Địa điểm làm việc 
  address = soup_job.find('div', {'class':'box-address'}).find('div', {'style':'margin-bottom: 10px'}).text.strip()
  address = re.split('\n', address)

  return [job_title, name_company, intro_company, size_company, address_company, Salary, NumCadidate, HinhThuc, level, Gender, Experiment, jd1, jd2, jd3, date, NganhNghe, address]

In [ ]:
def get_infor_company_TCV(url):
  content_company = requests.get(url).content
  soup_company = BeautifulSoup(content_company, 'html.parser')
  # Giới thiệu công ty
  intro_company = soup_company.find('div', {'class':'intro-content'}).text.strip()
  intro_company = re.sub('\n+', '. ', intro_company)
  intro_company = re.sub('\xa0', '. ', intro_company)
  # Quy mô công ty
  size_company = soup_company.find('div', {'class':'information-section'}).find_all('div', {'class':'value-block'})[1].text.strip()
  # Địa điểm công ty
  address_company = soup_company.find('div', {'class':'content-contact'}).find_all('div', {'class':'info-line'})[2].text.strip()
  return [intro_company, size_company, address_company]

In [ ]:
def crawl_job_data_TCV2(url):
  content_job = requests.get(url).content
  soup_job = BeautifulSoup(content_job, 'html.parser')
  # Tiêu đề tuyển dụng
  job_title = soup_job.find('h2', {'class':'title '}).text.strip()
  # Tên công ty
  name_company = soup_job.find('div', {'id':'company-name'}).text.strip()
  # Thông tin công ty
  url_company = soup_job.find('div', {'class':'row jobfair-header'}).find('a', href=True)['href']
  infor_company = get_infor_company_TCV(url_company)
  # Giới thiệu công ty
  intro_company = infor_company[0]
  # Quy mô công ty
  size_company = infor_company[1]
  # Địa điểm công ty
  address_company = infor_company[2]
  # Thông tin chung
  general = soup_job.find('div', {'class':'box-main'}).text.strip()
  general = re.sub('\n', ' ', general)
  general = re.split('   ', general)
  general = list(filter(lambda x: x != '', general))

  for i in range(len(general)):
    general[i] = general[i].strip()

  # Mức lương
  Salary = general[1]
  # Số lượng tuyển
  NumCadidate = general[3]
  # Hình thức làm việc
  HinhThuc = general[5]
  # Cấp bậc
  level = general[7]
  # Giới tính 
  Gender = general[9]
  # Kinh nghiệm
  Experiment = general[11]
  # Mô tả công việc
  JD1 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[0].text.strip()
  jd1 = re.sub('\n+', '. ', JD1)
  # Yêu cầu ứng viên
  JD2 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[1].text.strip()
  jd2 = re.sub('\n+', '. ', JD2)
  # Quyền lợi
  JD3 = soup_job.find('div', {'class':'job-data'}).find_all('div', {'class':'content-tab'})[2].text.strip()
  jd3 = re.sub('\n+', '. ', JD3)
  # Hạn nộp hồ sơ
  days = soup_job.find('span', {'class':'deadline'}).text.strip()
  days = int(re.findall(r'\d+', days)[0])

  today = datetime.date(2022, 12, 7)
  end_date = str(today + datetime.timedelta(days=days))
  end_date = re.sub(r'(\d{4})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', end_date)
  # Ngành nghề
  NganhNghe = soup_job.find('div', {'class':'section '}).find('div', {'class':'mb-20'}).text.strip()
  NganhNghe = re.split('\n+', NganhNghe)
  # Địa điểm làm việc 
  address = soup_job.find('div', {'class':'box-address bg-premium-hover'}).find('div').text.strip()
  address = re.split('\n', address)

  return [job_title, name_company, intro_company, size_company, address_company, Salary, NumCadidate, HinhThuc, level, Gender, Experiment, jd1, jd2, jd3, end_date, NganhNghe, address]

In [ ]:
url_job_TCV, job_title_TCV, name_company_TCV, intro_company_TCV, size_company_TCV, address_company_TCV, salary_TCV, numCadidate_TCV = [], [], [], [], [], [], [], []
HinhThuc_TCV, level_TCV, gerder_TCV, experiment_TCV, jd1_TCV, jd2_TCV, jd3_TCV, date_TCV, NganhNghe_TCV, address_TCV = [], [], [], [], [], [], [], [], [], []

for url in tqdm(list_url_job_TCV):
  try:
    job_data_TCV = crawl_job_data_TCV1(url)
    url_job_TCV.append(url)
    job_title_TCV.append(job_data_TCV[0])
    name_company_TCV.append(job_data_TCV[1])
    intro_company_TCV.append(job_data_TCV[2])
    size_company_TCV.append(job_data_TCV[3])
    address_company_TCV.append(job_data_TCV[4])
    salary_TCV.append(job_data_TCV[5])
    numCadidate_TCV.append(job_data_TCV[6])
    HinhThuc_TCV.append(job_data_TCV[7])
    level_TCV.append(job_data_TCV[8])
    gerder_TCV.append(job_data_TCV[9])
    experiment_TCV.append(job_data_TCV[10])
    jd1_TCV.append(job_data_TCV[11])
    jd2_TCV.append(job_data_TCV[12])
    jd3_TCV.append(job_data_TCV[13])
    date_TCV.append(job_data_TCV[14])
    NganhNghe_TCV.append(job_data_TCV[15])
    address_TCV.append(job_data_TCV[16])
  except:
    try:
      job_data_TCV = crawl_job_data_TCV2(url)
      url_job_TCV.append(url)
      job_title_TCV.append(job_data_TCV[0])
      name_company_TCV.append(job_data_TCV[1])
      intro_company_TCV.append(job_data_TCV[2])
      size_company_TCV.append(job_data_TCV[3])
      address_company_TCV.append(job_data_TCV[4])
      salary_TCV.append(job_data_TCV[5])
      numCadidate_TCV.append(job_data_TCV[6])
      HinhThuc_TCV.append(job_data_TCV[7])
      level_TCV.append(job_data_TCV[8])
      gerder_TCV.append(job_data_TCV[9])
      experiment_TCV.append(job_data_TCV[10])
      jd1_TCV.append(job_data_TCV[11])
      jd2_TCV.append(job_data_TCV[12])
      jd3_TCV.append(job_data_TCV[13])
      date_TCV.append(job_data_TCV[14])
      NganhNghe_TCV.append(job_data_TCV[15])
      address_TCV.append(job_data_TCV[16])
    except:
      # print(url)
      continue

100%|██████████| 9985/9985 [4:50:53<00:00,  1.75s/it]


### Save 

| Job Title | Name Company | Company Overview | Company Size | Company Address | Job Description | Job Requirements | Benefits | Job Address | Job Type | Gender | Number Cadidate | Career Level | Years of Experience | Salary | Submission Deadline | Industry |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|

In [ ]:
# Result
Dataset_TCV = pd.DataFrame({'URL Job': url_job_TCV,
                            'Job Title': job_title_TCV,
                            'Name Company': name_company_TCV,
                            'Company Overview': intro_company_TCV,
                            'Company Size': size_company_TCV,
                            'Company Address': address_company_TCV,
                            'Job Description': jd1_TCV,
                            'Job Requirements': jd2_TCV,
                            'Benefits': jd3_TCV,
                            'Job Address': address_TCV,
                            'Job Type': HinhThuc_TCV,
                            'Gender': gerder_TCV,
                            'Number Cadidate': numCadidate_TCV,
                            'Career Level': level_TCV,
                            'Years of Experience': experiment_TCV,
                            'Salary': salary_TCV,
                            'Submission Deadline': date_TCV,
                            'Industry': NganhNghe_TCV})

Dataset_TCV

,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499 nhân viên,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,"[- Hà Nội: 18 Tôn Thất Thuyết, Nam Từ Liêm]",Toàn thời gian,Không yêu cầu,1 người,Nhân viên,1 năm,9-11 triệu,05/01/2023,"[Hành chính / Văn phòng, Kinh doanh / Bán hàng..."
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499 nhân viên,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,"[- Hồ Chí Minh: 235 Lý Thường Kiệt, Phường 6, ...",Toàn thời gian,Không yêu cầu,2 người,Thực tập sinh,Không yêu cầu kinh nghiệm,Tới 1 triệu,01/01/2023,"[Công nghệ thông tin, IT phần mềm, Việc làm IT]"
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499 nhân viên,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,[- Hồ Chí Minh: Phú Nhuận],Toàn thời gian,Không yêu cầu,1 người,Trưởng/Phó phòng,Trên 5 năm,Thoả thuận,28/12/2022,"[Nhân sự, Quản lý điều hành, Xuất nhập khẩu]"
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99 nhân viên,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...","[- Đà Nẵng: 80 Đông Kinh Nghĩa Thục, Sơn Trà]",Toàn thời gian,Không yêu cầu,1 người,Giám đốc,3 năm,Thoả thuận,31/12/2022,"[Khách sạn / Nhà hàng, Thực phẩm / Đồ uống, Sp..."
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499 nhân viên,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,"[- Hồ Chí Minh: Gymaster 99 Phan Văn Hớn, Quậ...",Toàn thời gian,Không yêu cầu,2 người,Nhân viên,Không yêu cầu kinh nghiệm,6-12 triệu,06/01/2023,"[Hành chính / Văn phòng, Dịch vụ khách hàng, T..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9918,https://www.topcv.vn/viec-lam/nhan-vien-sales-...,Nhân Viên Sales Tour Nội Địa,Công ty TNHH Đầu tư thương mại và du lịch quốc...,Với niềm đam mê cháy bỏng dành cho nghề tổ chứ...,100-499 nhân viên,"29, Đoàn Thị Điểm, Quận Đống Đa, Hà Nội","Chủ động tìm kiếm khách hàng, Nhận yêu cầu đặ...",Hiểu biết các tuyến điểm là một lợi thế.Thành...,Mức lương thỏa thuận (theo năng lực) + Thưởng ...,"[- Hà Nội: 29 Đoàn Thị Điểm, Đống Đa]",Toàn thời gian,Không yêu cầu,3 người,Nhân viên,1 năm,Thoả thuận,06/01/2023,"[Kinh doanh / Bán hàng, Du lịch, Hành chính / ..."
9919,https://www.topcv.vn/viec-lam/chinese-speaking...,Chinese-Speaking Marketing Intern,CÔNG TY CỔ PHẦN BẢO HIỂM CÔNG NGHỆ MEDICI,,25-99 nhân viên,"Tầng 7, Tòa nhà Peakview, số 36 Hoàng Cầu, Phư...",Scrolling through & translating from Chinese s...,"Enjoying visual story-telling, creating meanin...","Class A office with coz

In [ ]:
Dataset_TCV.to_csv(PATH+'/job_data_TCV.csv', index = False)

## Crawl job data from [timviec365](https://timviec365.vn/tin-tuyen-dung-viec-lam.html)

In [ ]:
url_TV365 = 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html'

In [ ]:
TV365 = requests.get(url_TV365).content
print(TV365)

b'<!DOCTYPE html>\n<html lang="vi" prefix="og: https://ogp.me/ns#" class="no-js no-svg">\n<head>\n\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n  <title>Vi\xe1\xbb\x87c t\xc3\xacm ng\xc6\xb0\xe1\xbb\x9di m\xe1\xbb\x9bi nh\xe1\xba\xa5t t\xe1\xba\xa1i Timviec365.vn</title>\n  <meta itemprop="headline" content="Vi\xe1\xbb\x87c t\xc3\xacm ng\xc6\xb0\xe1\xbb\x9di m\xe1\xbb\x9bi nh\xe1\xba\xa5t t\xe1\xba\xa1i Timviec365.vn" />\n  <meta itemprop="description"  name="description" content="Th\xc3\xb4ng tin v\xe1\xbb\x81 vi\xe1\xbb\x87c t\xc3\xacm ng\xc6\xb0\xe1\xbb\x9di \xc4\x91\xc6\xb0\xe1\xbb\xa3c c\xe1\xba\xadp nh\xe1\xba\xadt th\xc6\xb0\xe1\xbb\x9dng xuy\xc3\xaan t\xe1\xba\xa1i timviec365.vn. T\xe1\xba\xa5t c\xe1\xba\xa3 c\xc3\xa1c tin tuy\xe1\xbb\x83n d\xe1\xbb\xa5ng t\xe1\xbb\xab c\xc3\xa1c c\xc3\xb4ng ty l\xe1\xbb\x9bn \xc4\x91\xe1\xbb\x81 \xc4\x91\xc6\xb0\xe1\xbb\xa3c ki\xe1\xbb\x83m duy\xe1\xbb\x87t r\xe1\xba\xa5t k\xc4\xa9."/>\n  <!-- <link rel="canonical" 

In [ ]:
soup_TV365 = BeautifulSoup(TV365, 'html.parser')
print(soup_TV365)

<!DOCTYPE html>

<html class="no-js no-svg" lang="vi" prefix="og: https://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Việc tìm người mới nhất tại Timviec365.vn</title>
<meta content="Việc tìm người mới nhất tại Timviec365.vn" itemprop="headline"/>
<meta content="Thông tin về việc tìm người được cập nhật thường xuyên tại timviec365.vn. Tất cả các tin tuyển dụng từ các công ty lớn đề được kiểm duyệt rất kĩ." itemprop="description" name="description"/>
<!-- <link rel="canonical" href="https://timviec365.vn/tin-tuyen-dung-viec-lam.html"/> -->
<link href="https://timviec365.vn/" rel="canonical"/>
<meta content="index,follow" name="robots"/>
<script type="application/ld+json">
  {
      "@context": "https://schema.org",
      "@type": "EmploymentAgency",
      "name": "Timviec365.vn",
      "alternateName": "Tìm Việc Làm & Tuyển Dụng Việc Làm Nhanh | timviec365.vn",
      "logo": "https://timviec365.vn/images/logo3.png",
      "image": "htt

### Crawl url page

In [ ]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_TV365(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page='+str(i))
  return list_url_page

In [ ]:
# Lấy danh sách các page có danh sách các công việc
last_page = soup_TV365.find('a', class_=' last')['href']
total_page = int(re.findall(r'\d+', last_page)[0])
list_url_page = get_url_page_TV365(total_page)
list_url_page

['https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=1',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=2',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=3',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=4',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=5',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=6',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=7',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=8',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=9',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=10',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=11',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=12',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=13',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=14',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=15',
 'https://timviec365.vn/tin-tuyen-dung-viec-lam.html?page=16',
 

### Crawl url job

In [ ]:
# Viết hàm lấy url job của các công việc
def get_url_job_TV365(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_job = []
  # Lấy nội dung của page
  Content = requests.get(url_page).content
  Soup = BeautifulSoup(Content, 'html.parser')
  # Lấy url job
  url_jobs = Soup.find('div', {'class': 'main_cate'}).find_all('a', href=True, class_='tag_th title_cate ')
  for job in url_jobs:
    list_url_job.append('https://timviec365.vn'+job['href'])
  
  return list_url_job

In [ ]:
list_url_job_TV365 = []
for url_page in tqdm(list_url_page):
  try:
    list_url_job_TV365.extend(get_url_job_TV365(url_page))
  except:
    print(f'{url_page} -> Error')

100%|██████████| 237/237 [08:56<00:00,  2.26s/it]


In [ ]:
list_url_job_TV365 = list(set(list_url_job_TV365))
len(list_url_job_TV365)

4737

In [ ]:
df_url_job_TV365 = pd.DataFrame({'URL Job': list_url_job_TV365})
df_url_job_TV365

,URL Job
0,https://timviec365.vn/nhan-vien-cham-soc-khach...
1,https://timviec365.vn/tuyen-chuyen-vien-kinh-d...
2,https://timviec365.vn/hr-p844931.html
3,https://timviec365.vn/chuyen-vien-tu-van-san-p...
4,https://timviec365.vn/nhan-vien-hanh-chinh-van...
...,...
4732,https://timviec365.vn/viec-lam-ban-thoi-gian-t...
4733,https://timviec365.vn/nhan-vien-tu-van-tin-dun...
4734,https://timviec365.vn/tuyen-nhan-vien-ki-thuat...
4735,https://timviec365.vn/tro-ly-chu-tich-hdqt-p84...


In [ ]:
df_url_job_TV365.to_csv(PATH+'/job_url_TV365.csv',index_label='ID')

### Crawl job data

| Job Title | Name Company | Company Overview | Company Size | Company Address | Job Description | Job Requirements | Benefits | Job Address | Job Type | Gender | Number Cadidate | Career Level | Years of Experience | Salary | Submission Deadline | Industry |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|

In [ ]:
def normalize_text(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  return text

In [ ]:
def get_infor_company_TV365(url):
  content_company = requests.get(url).content
  soup_company = BeautifulSoup(content_company, 'html.parser')
  # Giới thiệu công ty
  company_overview = soup_company.find('div', {'class':'left_com'}).text.strip()
  company_overview = normalize_text(company_overview)
  # Quy mô công ty
  company_size =  soup_company.find('div', {'class':'text_ntd_2'}).find_all('span')[1].text.strip()[8:]
  # Địa điểm công ty
  company_address = soup_company.find('div', {'class':'text_ntd_2'}).find_all('span')[0].text.strip()[9:]
  return [company_overview, company_size, company_address]

In [ ]:
def crawl_job_data_TV365(url):
  content_job = requests.get(url).content
  soup_job = BeautifulSoup(content_job, 'html.parser')
  # Tiêu đề tuyển dụng
  job_title = soup_job.find('div', {'class':'right_tit'}).find('h1').text.strip()
  # Tên công ty
  name_company = soup_job.find('a', {'class':'na_cty'}).text.strip()
  # Thông tin công ty
  url_company = 'https://timviec365.vn'+soup_job.find('a', {'class':'na_cty'})['href']
  infor_company = get_infor_company_TV365(url_company)
  # Giới thiệu công ty
  company_overview = infor_company[0]
  # Quy mô công ty
  company_size = infor_company[1]
  # Địa điểm công ty
  company_address = infor_company[2]
  # Mô tả công việc
  job_description = soup_job.find('div', {'class':'box_mota'}).text.strip()
  job_description = normalize_text(job_description)
  # Yêu cầu công việc
  job_requirements = soup_job.find('div', {'class':'box_yeucau'}).text.strip()
  job_requirements = normalize_text(job_requirements)
  # Quyền lợi
  benefits = soup_job.find('div', {'class':'box_quyenloi'}).text.strip()
  benefits = normalize_text(benefits)
  # Địa điểm làm việc 
  job_address = soup_job.find('p', {'class':'dd_tuyen'}).text.strip()[23:]
  job_address = normalize_text(job_address)
  # Hình thức làm việc
  job_type = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[2].text.strip()[22:]
  # Giới tính 
  gender = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[4].text.strip()[21:]
  # Số lượng tuyển
  num_cadidate = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[5].text.strip()[22:]
  # Cấp bậc
  career_level = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[0].text.strip()[11:]
  # Kinh nghiệm
  years_of_experience = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[1].text.strip()[15:]
  # Mức lương
  salary = soup_job.find_all('p', {'class':'dd_tuyen'})[-1].text.strip()[11:]
  # Hạn nộp hồ sơ
  submission_deadline = soup_job.find('div', {'class':'right_tit'}).find_all('span')[-1].text.strip()
  # Ngành nghề
  industry = soup_job.find('div', {'class':'box_tomtat_2'}).find_all('p')[-1].text.strip()[14:]
  industry = industry.strip()
  industry = re.sub('\n+', ', ', industry)
  

  return [job_title, name_company, company_overview, company_size, company_address, 
          job_description, job_requirements, benefits, job_address, job_type, gender, num_cadidate, 
          career_level, years_of_experience, salary, submission_deadline, industry]

In [ ]:
url_job_TV365, job_title_TV365, name_company_TV365, company_overview_TV365, company_size_TV365, company_address_TV365 = [], [], [], [], [], []
job_description_TV365, job_requirements_TV365, benefits_TV365, job_address_TV365, job_type_TV365, gender_TV365 = [], [], [], [], [], []
num_cadidate_TV365, career_level_TV365, years_of_experience_TV365, salary_TV365, submission_deadline_TV365, industry_TV365 = [], [], [], [], [], []

for url in tqdm(list_url_job_TV365):
  try:
    job_data_TV365 = crawl_job_data_TV365(url)
    url_job_TV365.append(url)
    job_title_TV365.append(job_data_TV365[0])
    name_company_TV365.append(job_data_TV365[1])
    company_overview_TV365.append(job_data_TV365[2])
    company_size_TV365.append(job_data_TV365[3])
    company_address_TV365.append(job_data_TV365[4])
    job_description_TV365.append(job_data_TV365[5])
    job_requirements_TV365.append(job_data_TV365[6])
    benefits_TV365.append(job_data_TV365[7])
    job_address_TV365.append(job_data_TV365[8])
    job_type_TV365.append(job_data_TV365[9])
    gender_TV365.append(job_data_TV365[10])
    num_cadidate_TV365.append(job_data_TV365[11])
    career_level_TV365.append(job_data_TV365[12])
    years_of_experience_TV365.append(job_data_TV365[13])
    salary_TV365.append(job_data_TV365[14])
    submission_deadline_TV365.append(job_data_TV365[15])
    industry_TV365.append(job_data_TV365[16])
  except:
    # print(url)
    continue

100%|██████████| 4737/4737 [5:21:24<00:00,  4.07s/it]


### Save 

In [ ]:
# Result
Dataset_TV365 = pd.DataFrame({'URL Job': url_job_TV365,
                            'Job Title': job_title_TV365,
                            'Name Company': name_company_TV365,
                            'Company Overview': company_overview_TV365,
                            'Company Size': company_size_TV365,
                            'Company Address': company_address_TV365,
                            'Job Description': job_description_TV365,
                            'Job Requirements': job_requirements_TV365,
                            'Benefits': benefits_TV365,
                            'Job Address': job_address_TV365,
                            'Job Type': job_type_TV365,
                            'Gender': gender_TV365,
                            'Number Cadidate': num_cadidate_TV365,
                            'Career Level': career_level_TV365,
                            'Years of Experience': years_of_experience_TV365,
                            'Salary': salary_TV365,
                            'Submission Deadline': submission_deadline_TV365,
                            'Industry': industry_TV365})

Dataset_TV365

,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://timviec365.vn/nhan-vien-cham-soc-khach...,Nhân viên chăm sóc khách hàng - hỗ trợ KOls Sh...,Công Ty Trách Nhiệm Hữu Hạn Việt Mỹ - Ssu,Mô tả công ty Văn hoá của Việt Mỹ SSU được xây...,Đang cập nhật,"82 Thăng Long, Phường 4, Quận Tân Bình",Mô tả công việc Liên hệ/ gửi email thông báo c...,Yêu cầu công việcNam/nữ tốt nghiệp THPT hoặc c...,Quyền lợi được hưởngThu nhập: 6.000.000 – 8.00...,Hồ Chí Minh,Toàn thời gian cố định,Không yêu cầu,5 ứng viên,Nhân viên,Không yêu cầu,Từ 6.000.000 VNĐ Đến 8.000.000 VNĐ,15/12/2022,nhân viên chăm sóc khách hàng
1,https://timviec365.vn/tuyen-chuyen-vien-kinh-d...,Tuyển chuyên viên kinh doanh,Công ty cổ phần Tập đoàn Việt Ba,Mô tả công ty Vietba Group được thành lập vào ...,Từ 25 - 99 nhân viên,"Tầng 3, Tòa nhà Toserco 273 Kim Mã, Ba Đình, H...",Mô tả công việc 1. Tìm kiếm khách hàng mới - C...,Yêu cầu công việc- Ưu tiên tốt nghiệp chuyên n...,Quyền lợi được hưởng-\tMôi trường làm việc năn...,Hà Nội,Toàn thời gian cố định,Không yêu cầu,2 ứng viên,Nhân viên,Hơn 2 năm kinh nghiệm,Từ 15.000.000 VNĐ,30/12/2022,chuyên viên kinh doanh
2,https://timviec365.vn/hr-p844931.html,HR,Công Ty TNHH Pink Venom,Mô tả công ty vào r biết,Đang cập nhật,Định Công,Mô tả công việc HR,Yêu cầu công việcTrưởng phòng,Quyền lợi được hưởngQuản lý,Hà Nội,Toàn thời gian cố định,0,1 ứng viên,Trưởng phòng,Hơn 2 năm kinh nghiệm,15 - 20 triệu,11/12/2022,Hành chính - Văn phòng
3,https://timviec365.vn/chuyen-vien-tu-van-san-p...,Chuyên viên tư vấn sẩn phẩm trực tiếp tại văn ...,Công Ty TNHH Cà Phê Việt Nam,Mô tả công ty,Đang cập nhật,"17 Tân Hương, Phường Tân Quý","Mô tả công việc - Cung cấp, hỗ trợ khách hàng ...",Yêu cầu công việc- Tuổi: 22 - 35 tuổi - Phân t...,"Quyền lợi được hưởng- Thu nhập hấp dẫn, Môi tr...",Hồ Chí Minh,Toàn thời gian cố định,Không yêu cầu,2 ứng viên,Nhân viên,Không yêu cầu,Từ 10.000.000 VNĐ,25/12/2022,nhân viên tư vấn bán hàng
4,https://timviec365.vn/nhan-vien-hanh-chinh-van...,nhân viên hành chính văn phòng,Cong Ty Co Phan Duoc Pham Bao Phuong,Mô tả công ty CÔNG TY CỔ PHẦN DƯỢC PHẨM BẢO PH...,Đang cập nhật,Di An Binh Duong,Mô tả công việc Mô tả công việc-thực hiện đề x...,Yêu cầu công việcYêu cầu công việcTốt nghiệp đ...,Quyền lợi được hưởngQuyền lợi được hưởngLương ...,Bình Dương,Toàn thời gian cố định,Nữ,2 ứng viên,Nhân viên,Hơn 1 năm kinh nghiệm,7 - 10 triệu,31/01/2023,nhân viên hành chính văn phòng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả công ty Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",Mô tả công việc - Đối tượng: phù hợp với tất c...,Yêu cầu công việcBẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY...,Quyền lợi được hưởng Các Phúc Lợi Dành Cho Bạn...,Đắk Lắk,Bán thời gian,Không yêu cầu,8 ứng viên,Nhân viên,Không yêu cầu,7 - 10 triệu,29/12/2028,Sinh viên làm thêm
4707,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,Mô tả công ty,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",Mô tả công việc 1.Mô tả công việc: - Sử dụng h...,Yêu cầu công việc- Tốt nghiệp THPT - Tuổi từ 1...,Quyền lợi được hưởng- Lương cơ bản: 5.100.000/...,Hồ Chí Minh,Toàn thời gian cố định,Không yêu cầu,100 ứng viên,Nhân viên,Không yêu cầu,5 - 7 triệu,30/12/2022,nhân viên tín dụng ngân hàng
4708,https://timviec365.vn/tuyen-nhan-vien-ki-thuat...,tuyển nhân viên kỹ thuật,Công Ty Tnhh Mtv Az Company,Mô tả công ty công ty chúng tôi chuyên cung cấ...,Đang cập nhật,114 B15,Mô tả công việc công ty tuyển nhân viên bảo vệ...,"Yêu cầu công việcyêu cầu ứng viên nam, tuổi từ...",Quyền lợi được hưởngl

In [ ]:
Dataset_TV365.to_csv(PATH+'job_data_TV365.csv', index = False)

## Crawl user data from [timviec365](https://timviec365.vn/nguoi-tim-viec.html)

In [ ]:
url_user_TV365 = 'https://timviec365.vn/nguoi-tim-viec.html'

In [ ]:
user_TV365 = requests.get(url_user_TV365).content
print(user_TV365)

b'<!DOCTYPE html>\n<html lang="vi">\n<head>\n\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n  <title>Ng\xc6\xb0\xe1\xbb\x9di t\xc3\xacm vi\xe1\xbb\x87c m\xe1\xbb\x9bi nh\xe1\xba\xa5t - Xem H\xe1\xbb\x93 s\xc6\xa1,CV \xe1\xbb\xa9ng vi\xc3\xaan mi\xe1\xbb\x85n ph\xc3\xad |Timviec365</title>\n  <meta itemprop="headline" content="Ng\xc6\xb0\xe1\xbb\x9di t\xc3\xacm vi\xe1\xbb\x87c m\xe1\xbb\x9bi nh\xe1\xba\xa5t - Xem H\xe1\xbb\x93 s\xc6\xa1,CV \xe1\xbb\xa9ng vi\xc3\xaan mi\xe1\xbb\x85n ph\xc3\xad |Timviec365" />\n  <meta itemprop="description"  name="description" content="Ngu\xe1\xbb\x9di t\xc3\xacm vi\xe1\xbb\x87c m\xe1\xbb\x9bi nh\xe1\xba\xa5t, ch\xe1\xba\xa5t l\xc6\xb0\xe1\xbb\xa3ng nh\xe1\xba\xa5t. H\xc3\xa0ng tr\xc4\x83m ngh\xc3\xacn h\xe1\xbb\x93 s\xc6\xa1 \xe1\xbb\xa9ng vi\xc3\xaan tr\xc3\xaan to\xc3\xa0n qu\xe1\xbb\x91c \xc4\x91\xc6\xb0\xe1\xbb\xa3c c\xe1\xba\xadp nh\xe1\xba\xadt \xc4\x91\xe1\xba\xa7y \xc4\x91\xe1\xbb\xa7 t\xe1\xba\xa1i \xc4\x91\xe1\xbb\x8

In [ ]:
soup_user_TV365 = BeautifulSoup(user_TV365, 'html.parser')
print(soup_user_TV365)

<!DOCTYPE html>

<html lang="vi">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Người tìm việc mới nhất - Xem Hồ sơ,CV ứng viên miễn phí |Timviec365</title>
<meta content="Người tìm việc mới nhất - Xem Hồ sơ,CV ứng viên miễn phí |Timviec365" itemprop="headline"/>
<meta content="Nguời tìm việc mới nhất, chất lượng nhất. Hàng trăm nghìn hồ sơ ứng viên trên toàn quốc được cập nhật đầy đủ tại địa chỉ Timviec365.vn" itemprop="description" name="description"/>
<link href="https://timviec365.vn/nguoi-tim-viec.html" rel="canonical"/>
<meta content="index,follow" name="robots"/>
<script type="application/ld+json">
  {
      "@context": "https://schema.org",
      "@type": "EmploymentAgency",
      "name": "Timviec365.vn",
      "alternateName": "Tìm Việc Làm & Tuyển Dụng Việc Làm Nhanh | timviec365.vn",
      "logo": "https://timviec365.vn/images/logo3.png",
      "image": "https://timviec365.vn/images/logo3.png",
      "description": "Tìm việc 365 cung cấp 

### Crawl url page

In [ ]:
# Viết hàm lấy url page chứa danh sách các công việc
def get_url_page_user_TV365(num_page):
  list_url_page = []
  for i in range(1,num_page+1):
    list_url_page.append('https://timviec365.vn/nguoi-tim-viec.html?page='+str(i))
  return list_url_page

In [ ]:
# Lấy danh sách các page có danh sách các công việc
# last_page = soup_user_TV365.find('a', class_=' last')['href']
total_page = 1550
list_url_page = get_url_page_user_TV365(total_page)
list_url_page

['https://timviec365.vn/nguoi-tim-viec.html?page=1',
 'https://timviec365.vn/nguoi-tim-viec.html?page=2',
 'https://timviec365.vn/nguoi-tim-viec.html?page=3',
 'https://timviec365.vn/nguoi-tim-viec.html?page=4',
 'https://timviec365.vn/nguoi-tim-viec.html?page=5',
 'https://timviec365.vn/nguoi-tim-viec.html?page=6',
 'https://timviec365.vn/nguoi-tim-viec.html?page=7',
 'https://timviec365.vn/nguoi-tim-viec.html?page=8',
 'https://timviec365.vn/nguoi-tim-viec.html?page=9',
 'https://timviec365.vn/nguoi-tim-viec.html?page=10',
 'https://timviec365.vn/nguoi-tim-viec.html?page=11',
 'https://timviec365.vn/nguoi-tim-viec.html?page=12',
 'https://timviec365.vn/nguoi-tim-viec.html?page=13',
 'https://timviec365.vn/nguoi-tim-viec.html?page=14',
 'https://timviec365.vn/nguoi-tim-viec.html?page=15',
 'https://timviec365.vn/nguoi-tim-viec.html?page=16',
 'https://timviec365.vn/nguoi-tim-viec.html?page=17',
 'https://timviec365.vn/nguoi-tim-viec.html?page=18',
 'https://timviec365.vn/nguoi-tim-vie

### Crawl url user

In [ ]:
# Viết hàm lấy url user
def get_url_user_TV365(url_page):
  # Khởi tạo danh sách url các công việc
  list_url_user = []
  # Lấy nội dung của page
  Content = requests.get(url_page).content
  Soup = BeautifulSoup(Content, 'html.parser')
  # Lấy url user
  url_users = Soup.find('div', {'id': 'box_nd_uv'}).find_all('a', href=True)
  for i in range(0,len(url_users),5):
    list_url_user.append(f"https://timviec365.vn{url_users[i]['href']}")
  
  return list_url_user

In [ ]:
list_url_user_TV365 = []
for url_page in tqdm(list_url_page):
  try:
    list_url_user_TV365.extend(get_url_user_TV365(url_page))
  except:
    print(f'{url_page} -> Error')

len(list_url_user_TV365)

100%|██████████| 1550/1550 [1:04:15<00:00,  2.49s/it]


45358

In [ ]:
list_url_user_TV365 = list(set(list_url_user_TV365))
len(list_url_user_TV365)

45355

In [ ]:
df_url_user_TV365 = pd.DataFrame({'URL User': list_url_user_TV365})
df_url_user_TV365

,URL User
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...
1,https://timviec365.vn/ung-vien/nguyen-thi-hong...
2,https://timviec365.vn/ung-vien/le-hong-phu-uv9...
3,https://timviec365.vn/ung-vien/nguyen-chau-uv9...
4,https://timviec365.vn/ung-vien/dang-tran-cong-...
...,...
45350,https://timviec365.vn/ung-vien/duong-minh-hoan...
45351,https://timviec365.vn/ung-vien/to-thanh-quang-...
45352,https://timviec365.vn/ung-vien/nguyen-ba-quang...
45353,https://timviec365.vn/ung-vien/nguyen-thi-hung...


In [ ]:
df_url_user_TV365.to_csv(PATH+'/user_url_TV365.csv',index_label='ID')

### Crawl user data

| UserID | User Name | Industry | Desired Job | Workplace Desired | Current Place of Residence | Province | District | Desired Salary | Gender | Marriage | Date of Birth | Work Experience | Career Goals | Personal Skills |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|

In [ ]:
def normalize_text(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  return text

In [ ]:
def crawl_user_data_TV365(url):
  content_user = requests.get(url).content
  soup_user = BeautifulSoup(content_user, 'html.parser')
  # ID của ứng viên
  userid = soup_user.find('div', {'class': 'box_ctuv_1'}).find('p', {'title': 'Mã hồ sơ'}).text
  userid = re.findall(r'\d+', userid)[0]
  # Tên của ứng viên
  user_name = soup_user.find('div', {'class': 'uvv_text'}).find('h1').text
  # Ngành nghề -> Có thể null
  industrys = soup_user.find('div', {'class': 'box_ctuv_4'}).find('p').find_all('span')
  industry = []
  for i in industrys:
    industry.append(i.text)
  # Công việc mong muốn
  desired_job = soup_user.find('div', {'class': 'box_ctuv_4'}).find_all('p')[1].find('b').text
  # Nơi mong muốn làm việc -> Có thể null
  workplace_desireds = soup_user.find('div', {'class': 'box_ctuv_4'}).find_all('p')[2].find_all('span')
  workplace_desired = []
  for i in workplace_desireds:
    workplace_desired.append(i.text)
  # Chỗ ở hiện tại 
  current_place_of_residence = soup_user.find('div', {'class': 'box_ctuv_4'}).find_all('p')[3].text[16:]
  # Tỉnh thành -> Có thể null
  try:
    province = soup_user.find('div', {'class': 'box_ctuv_4'}).find_all('p')[4]
    province = province.text[12:]
  except:
    province = '?'
  # Quận huyện -> Có thể null
  try:
    district = soup_user.find('div', {'class': 'box_ctuv_4'}).find_all('p')[5].text[12:]
  except:
    district = '?'
  # Mức lương mong muốn
  desired_salary = soup_user.find('div', {'class': 'box_ctuv_4'}).find('p', {'title': 'Mức lương'}).find('span').text
  # Giới tính -> Có thể null
  try:
    gender = soup_user.find('div', {'class': 'box_ctuv_1'}).find('p', {'title': 'giới tính'})
    gender = normalize_text(gender.text.strip())[11:]
  except:
    gender = '?'
  # Hôn nhân -> Có thể null
  try:
    marriage = soup_user.find('div', {'class': 'box_ctuv_1'}).find('p', {'title': 'Hôn nhân'})
    marriage = normalize_text(marriage.text.strip())[10:]
  except:
    marriage = '?'
  # Ngày sinh -> Có thể null
  try:
    date_of_birth = soup_user.find('div', {'class': 'box_ctuv_1'}).find('p', {'title': 'ngày sinh'})
    date_of_birth = normalize_text(date_of_birth.text.strip())[11:]
  except:
    date_of_birth = '?'
  # Kinh nghiệm làm việc
  work_experience = soup_user.find('div', {'class': 'box_ctuv_1'}).find_all('p')[-1].find('span').text
  # Mục tiêu nghề nghiệp
  # career_goals = soup_user.find('div', {'class': 'content_cv_uv'}).find_all('div', {'class':'box_hs_detail'})[1].find('div', {'class':'s-text-spa'}).text
  # Kĩ năng bản thân
  return [userid, user_name, industrys, desired_job, workplace_desired, current_place_of_residence, province, district, desired_salary, gender, marriage, date_of_birth, work_experience]

In [ ]:
url_user, userid, user_name, industrys, desired_job, workplace_desired, current_place_of_residence = [], [], [], [], [], [], []
province, district, desired_salary, gender, marriage, date_of_birth, work_experience = [], [], [], [], [], [], []

for url in tqdm(list_url_user_TV365):
  try:
    user_data_TV365 = crawl_user_data_TV365(url)
    url_user.append(url)
    userid.append(user_data_TV365[0])
    user_name.append(user_data_TV365[1])
    industrys.append(user_data_TV365[2])
    desired_job.append(user_data_TV365[3])
    workplace_desired.append(user_data_TV365[4])
    current_place_of_residence.append(user_data_TV365[5])
    province.append(user_data_TV365[6])
    district.append(user_data_TV365[7])
    desired_salary.append(user_data_TV365[8])
    gender.append(user_data_TV365[9])
    marriage.append(user_data_TV365[10])
    date_of_birth.append(user_data_TV365[11])
    work_experience.append(user_data_TV365[12])
  except:
    # print(url)
    continue

 17%|█▋        | 7516/45355 [5:36:36<39:39:49,  3.77s/it]

### Save 

In [ ]:
# Result
Dataset_user = pd.DataFrame({'URL User': url_user,
                            'UserID': userid,
                            'User Name': user_name,
                            'Industry': industrys,
                            'Desired Job': desired_job,
                            'Workplace Desired': workplace_desired,
                            'Current Place of Residence': current_place_of_residence,
                            'Province': province,
                            'District': district,
                            'Desired Salary': desired_salary,
                            'Gender': gender,
                            'Marriage': marriage,
                            'Date of Birth': date_of_birth,
                            'Work Experience': work_experience})

Dataset_user

In [ ]:
Dataset_user.to_csv(PATH+'user_data_TV365.csv', index = False)